In [1]:
import pandas as pd
import os
import requests

In [2]:
FOURSQUARE_KEY = os.environ.get('FOURSQUARE_KEY')
YELP_KEY = os.environ.get('YELP_KEY')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
bike_stations = pd.read_csv("../data/greenbikeslc.csv")
bike_stations_sample = bike_stations.head()
bike_stations_sample

,short_id,name,address,latitude,longitude,empty_slots,free_bikes
0,4591,EcoBox Station @ 580S 500W,580 S 500 W,40.75689,-111.90523,7,5
1,2737,Energy Development Station @ 136S Main Street,136 S Main St,40.76624,-111.89124,17,2
2,2255,City Weekly Station @ 353S Main Street,353 S Main St,40.76142,-111.89091,13,10
3,7607,500N 300W Station,500 N 300 W,40.78045,-111.89945,15,3
4,4527,CW Urban Station @ theBeverly,63 S 600 W,40.76816,-111.90809,11,5


# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [37]:
def get_everything_fsq(bike_id, latitude, longitude, limit):
    url = "https://api.foursquare.com/v3/places/search"
    headers = {"Accept": "application/json"}
    headers["Authorization"] = FOURSQUARE_KEY
    params_dict = {
        "v": "20231010",
        "ll": f"{latitude},{longitude}",
        "radius": 1000,
        "limit": limit,
        "sort": "DISTANCE"
    }
    response = requests.get(url, headers=headers, params=params_dict)
    inner = response.json().get('results', {})
    df = pd.DataFrame(inner)   

    return pd.DataFrame(df)

In [38]:
def everything_near_bikes(dataframe, limit):
    copy_1 = dataframe.copy()
    copy_1['venues'] = dataframe.apply(
        lambda row: get_everything_fsq(row['short_id'], row['latitude'], row['longitude'], limit), axis=1)
    copy_2 = copy_1.copy()
    copy_2['venues_list'] = copy_1['venues'].apply(lambda df: df.to_dict('records'))
    copy_2 = copy_2.drop(columns=['venues']).explode('venues_list').reset_index(drop=True)
    copy_2 = pd.concat([copy_2.drop(columns=['venues_list']), copy_2['venues_list'].apply(pd.Series)], axis=1)
    copy_2 = copy_2.drop(columns=['name', 'address', 'latitude', 'longitude', 'empty_slots', 'free_bikes'])
    copy_2
    return copy_2

In [39]:
everything_near_bikes(bike_stations_sample, 2)

,short_id,fsq_id,categories,chains,closed_bucket,distance,geocodes,link,location,related_places,timezone
0,4591,5026151c5eda48fad31b09c7,"[{'id': 17033, 'name': 'Department Store', 'sh...",[],Unsure,160,"{'main': {'latitude': 40.755196, 'longitude': ...",/v3/places/5026151c5eda48fad31b09c7,"{'address': '537 W 600 S', 'census_block': '49...",{},America/Denver
1,4591,57fc01ae498e0eba9cfc0eae,"[{'id': 17012, 'name': 'Motorcycle Dealership'...",[],VeryLikelyOpen,270,"{'main': {'latitude': 40.759301, 'longitude': ...",/v3/places/57fc01ae498e0eba9cfc0eae,"{'address': '454 S 500 W', 'census_block': '49...",{},America/Denver
2,2737,4aef5fccf964a520f1d721e3,"[{'id': 13007, 'name': 'Beer Garden', 'short_n...",[],Unsure,30,"{'main': {'latitude': 40.766398, 'longitude': ...",/v3/places/4aef5fccf964a520f1d721e3,"{'address': '128 S Main St', 'census_block': '...",{},America/Denver
3,2737,57d5e241498e5a499911eaca,"[{'id': 10039, 'name': 'Music Venue', 'short_n...",[],Unsure,59,"{'main': {'latitude': 40.766391, 'longitude': ...",/v3/places/57d5e241498e5a499911eaca,"{'address': '131 S Main St', 'census_block': '...",{'children': []},America/Denver
4,2255,59b5a1d631fd145ab6d42e86,"[{'id': 10003, 'name': 'Arcade', 'short_name':...",[],Unsure,50,"{'main': {'latitude': 40.761017, 'longitude': ...",/v3/places/59b5a1d631fd145ab6d42e86,"{'address': '5 E 400 S', 'census_block': '4903...",{},America/Denver
5,2255,594589aef62f2b6488e63c14,"[{'id': 13024, 'name': 'Whisky Bar', 'short_na...",[],Unsure,73,"{'main': {'latitude': 40.762086, 'longitude': ...",/v3/places/594589aef62f2b6488e63c14,"{'address': '325 S Main St', 'census_block': '...",{},America/Denver
6,7607,635450f1f511005b3db79996,"[{'id': 13033, 'name': 'Bubble Tea Shop', 'sho...",[],LikelyOpen,73,"{'main': {'latitude': 40.780955, 'longitude': ...",/v3/places/635450f1f511005b3db79996,"{'address': '535 N 300 W', 'census_block': '49...",{},America/Denver
7,7607,4b7d4d30f964a52028b72fe3,"[{'id': 17029, 'name': 'Convenience Store', 's...",[{'id': 'ab4ba0c0-d68a-012e-5619-003048cad9da'...,VeryLikelyOpen,74,"{'drop_off': {'latitude': 40.780217, 'longitud...",/v3/places/4b7d4d30f964a52028b72fe3,"{'address': '515 N 300 W', 'census_block': '49...",{},America/Denver
8,4527,4c9a765f7421b7132df7fe13,"[{'id': 10037, 'name': 'Concert Hall', 'short_...",[],LikelyOpen,130,"{'main': {'latitude': 40.767722, 'longitude': ...",/v3/places/4c9a765f7421b7132df7fe13,"{'address': '536 W 100 S', 'census_block': '49...",{},America/Denver
9,4527,52f6fe8c11d2260115d77794,"[{'id': 13011, 'name': 'Gay Bar', 'short_name'...",[],Unsure,143,"{'main': {'latitude': 40.766933, 'longitude': ...",/v3/places/52f6fe8c11d2260115d77794,"{'address': '102 S 600 W', 'census_block': '49...",{},America/Denver


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [67]:
def get_venues_fsq_filter(bike_id, latitude, longitude, limit):
    url = "https://api.foursquare.com/v3/places/search"
    headers = {"Accept": "application/json"}
    headers["Authorization"] = FOURSQUARE_KEY
    params_dict = {
        "v": "20231010",
        "ll": f"{latitude},{longitude}",
        "radius": 1000,
        "categories": "13003",
        "limit": limit,
        "sort": "DISTANCE",
        "fields": "fsq_id,name,location,categories,rating,price,distance,stats"
    }
    response = requests.get(url, headers=headers, params=params_dict)
    inner = response.json().get('results', {})
    df = pd.DataFrame(inner)
    df.rename(columns={'name': 'venue_name'}, inplace=True)

    return pd.DataFrame(df)

In [ ]:
get_venues_fsq_filter(4545, 40.760779, -111.891047, 2)

In [ ]:
def nearby_venues_fsq(dataframe, limit):
    copy_1 = dataframe.copy()
    copy_1['venues'] = dataframe.apply(
        lambda row: get_venues_fsq_filter(row['short_id'], row['latitude'], row['longitude'], limit), axis=1)
    copy_2 = copy_1.copy()
    copy_2['venues_list'] = copy_1['venues'].apply(lambda df: df.to_dict('records') if df is not None else [])
    copy_2 = copy_2.drop(columns=['venues']).explode('venues_list').reset_index(drop=True)
    copy_2 = pd.concat([copy_2.drop(columns=['venues_list']), copy_2['venues_list'].apply(pd.Series)], axis=1)
    copy_2 = copy_2.drop(columns=['name','address', 'latitude', 'longitude', 'empty_slots', 'free_bikes'])
    info1 =  copy_2[['short_id', 'fsq_id', 'distance', 'price', 'rating', 'venue_name']]
    info2 = pd.json_normalize(copy_2['categories'])
    info2_cleaned =  pd.DataFrame(info2[0].apply(lambda x: x['name'] if x is not None else ''))
    info2_cleaned.columns = ['category']
    info3 = pd.json_normalize(copy_2['location'])
    info3_cleaned = pd.DataFrame(info3[['address']])
    info4_cleaned = pd.DataFrame(copy_2['stats'].apply(lambda x: x.get('total_ratings') if isinstance(x, dict) else ''))
    info4_cleaned.columns = ['total_ratings']
    info_merge = pd.concat([info1, info2_cleaned, info3_cleaned, info4_cleaned], axis=1)
    info_merge = info_merge.iloc[:, [0,1,5,6,7,2,3,4,8]]
    info_merge.rename(columns={'short_id': 'bike_station_id'}, inplace=True)
    return info_merge

nearby_venues_fsq(bike_stations_sample, 2)

Put your parsed results into a DataFrame

In [80]:
fsq_dataframe = nearby_venues_fsq(bike_stations, 50)

In [35]:
fsq_dataframe

,bike_station_id,fsq_id,venue_name,category,address,distance,price,rating,total_ratings
0,4591,52cd757b11d2a691812f93fe,Paint Nite,Bar,533 S 500 W,90.0,2.0,NaN,NaN
1,4591,4ad4f28ff964a52036fd20e3,Club Orange,Bar,533 S 500 W,90.0,2.0,NaN,NaN
2,4591,62a2127ce73a143f3f4501ca,Slackwater,Beer Bar,684 S 500 W,267.0,1.0,NaN,NaN
3,4591,54c3def2498ef58b7043770e,The Fallout,Night Club,625 S 600 W,268.0,NaN,NaN,NaN
4,4591,5a21c7e4ee712062bf95eab3,Kiitos Brewing,Bar,608 W 700 S,378.0,2.0,7.8,29.0
...,...,...,...,...,...,...,...,...,...
1918,3423,52cd757b11d2a691812f9417,Paint Nite,Bar,275 S 200 W,655.0,2.0,NaN,NaN
1919,3423,4ae0f334f964a520f98321e3,Gracie's,Sports Bar,326 S West Temple,656.0,2.0,7.2,259.0
1920,3423,4d12ea3ed0fda093f797af57,Graffiti Bar,Bar,342 S State St,658.0,2.0,NaN,NaN
1921,3423,4f32cfb619836c91c7fd4bc4,Club Naked,Gay Bar,326 S West Temple,666.0,2.0,NaN,NaN


In [82]:
#fsq_dataframe.to_csv("../data/fsq_dataframe.csv", index=False)
#only uncomment the above line if you want to overwrite the csv file

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [40]:
def get_everything_yelp(bike_id, latitude, longitude, limit):
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {YELP_KEY}"
    }
    params_dict = {
        'latitude': latitude,
        'longitude': longitude,
        "radius": 1000,
        "limit": limit,
        "sort_by": "distance"
    }
    response = (requests.get(url, headers=headers, params=params_dict)).json()
    ready = pd.json_normalize(response['businesses'])
    return response

In [41]:
def everything_near_bikes_yelp(dataframe, limit):
    copy_1 = dataframe.copy()
    copy_1['venues'] = dataframe.apply(
        lambda row: get_everything_yelp(row['short_id'], row['latitude'], row['longitude'], limit), axis=1)
    copy_2 = copy_1.copy()
    copy_2['venues_list'] = copy_1['venues'].apply(lambda df: df['businesses'] if 'businesses' in df else [])
    copy_2 = copy_2.drop(columns=['venues']).explode('venues_list').reset_index(drop=True)
    copy_2 = pd.concat([copy_2.drop(columns=['venues_list']), copy_2['venues_list'].apply(pd.Series)], axis=1)
    copy_2 = copy_2.drop(columns=['address', 'latitude', 'longitude', 'empty_slots', 'free_bikes'])
    copy_2
    return copy_2

In [42]:
everything_near_bikes_yelp(bike_stations_sample, 2)

,short_id,name,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,business_hours,attributes,price
0,4591,EcoBox Station @ 580S 500W,w11Ftny5iLK-Ozu2bb3URQ,liyas-cafe-slc-salt-lake-city,liyas cafe slc,,False,https://www.yelp.com/biz/liyas-cafe-slc-salt-l...,0,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",0.0,"{'latitude': 40.75514, 'longitude': -111.90569}","[delivery, pickup]","{'address1': '650 S 500th W', 'address2': None...",,,201.530488,[],"{'business_temp_closed': None, 'menu_url': 'ht...",NaN
1,4591,EcoBox Station @ 580S 500W,YxRCBsKiiJ7K3mJeIloO0Q,rimini-coffee-salt-lake-city,Rimini Coffee,https://s3-media3.fl.yelpcdn.com/bphoto/UKV6Wl...,False,https://www.yelp.com/biz/rimini-coffee-salt-la...,12,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.7,"{'latitude': 40.75749, 'longitude': -111.90282}",[delivery],"{'address1': '532 S 400th W', 'address2': '', ...",+18015391210,(801) 539-1210,205.245339,"[{'open': [{'is_overnight': False, 'start': '0...","{'business_temp_closed': None, 'menu_url': Non...",$$
2,2737,Energy Development Station @ 136S Main Street,5eDVL2I_97GgRlQ7G4CWEw,antica-roma-salt-lake-city,Antica Roma,https://s3-media4.fl.yelpcdn.com/bphoto/RUbHxm...,False,https://www.yelp.com/biz/antica-roma-salt-lake...,237,"[{'alias': 'italian', 'title': 'Italian'}]",3.8,"{'latitude': 40.76635, 'longitude': -111.89137}","[delivery, pickup]","{'address1': '132 S Main St', 'address2': '', ...",+18015320500,(801) 532-0500,10.476260,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': 'ht...",$$
3,2737,Energy Development Station @ 136S Main Street,m-7rn9RWSK4Ep8hjilyPqQ,beerhive-pub-salt-lake-city,Beerhive Pub,https://s3-media1.fl.yelpcdn.com/bphoto/lmOPVN...,False,https://www.yelp.com/biz/beerhive-pub-salt-lak...,504,"[{'alias': 'pubs', 'title': 'Pubs'}]",4.0,"{'latitude': 40.7664419836217, 'longitude': -1...",[delivery],"{'address1': '128 S Main St', 'address2': '', ...",+18013644268,(801) 364-4268,31.412682,"[{'open': [{'is_overnight': True, 'start': '12...","{'business_temp_closed': None, 'menu_url': Non...",$$
4,2255,City Weekly Station @ 353S Main Street,AqBFC8FO4d8ZgvzxgmG0Yw,aker-restaurant-and-lounge-salt-lake-city,Aker Restaurant & Lounge,https://s3-media1.fl.yelpcdn.com/bphoto/UTP-pF...,False,https://www.yelp.com/biz/aker-restaurant-and-l...,27,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.4,"{'latitude': 40.76156444515171, 'longitude': -...",[],"{'address1': '9 Exchange Pl', 'address2': '', ...",+13852954077,(385) 295-4077,33.917804,"[{'open': [{'is_overnight': False, 'start': '1...","{'business_temp_closed': None, 'menu_url': Non...",NaN
5,2255,City Weekly Station @ 353S Main Street,MfWhf5QUkIGa0K71ZDALXQ,alibi-bar-and-place-salt-lake-city,Alibi Bar & Place,https://s3-media1.fl.yelpcdn.com/bphoto/taRNma...,False,https://www.yelp.com/biz/alibi-bar-and-place-s...,69,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.3,"{'latitude': 40.76105, 'longitude': -111.89083}",[delivery],"{'address1': '369 Main St', 'address2': '', 'a...",+13852590616,(385) 259-0616,42.880532,"[{'open': [{'is_overnight': True, 'start': '16...","{'business_temp_closed': None, 'menu_url': Non...",NaN
6,7607,500N 300W Station,zgltHNNm3BHcE7Nys5KwHw,7-eleven-salt-lake-city-20,7-Eleven,https://s3-media2.fl.yelpcdn.com/bphoto/w9gtWo...,False,https://www.yelp.com/biz/7-eleven-salt-lake-ci...,8,"[{'alias': 'convenience', 'title': 'Convenienc...",1.8,"{'latitude': 40.780673447643, 'longitude': -11...","[delivery, pickup]","{'address1': '515 North 300 W', 'address2': No...",+18015217147,(801) 521-7147,74.984003,"[{'open': [{'is_overnight': True, 'start': '00...","{'business_temp_closed': None, 'menu_url': Non...",$
7,7607,500N 300W Station,s099WqfiTaanFF5Ar3KHUA,marmalade-brunch-house-salt-lake-city,Marmalade Brunch House,https://s3-media2.fl.yelpcdn.com/bphoto/HKD9tn...,False,https://www.yelp.com/biz/marmalade

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
def get_venues_yelp(bike_id, latitude, longitude, limit):
    url = "https://api.yelp.com/v3/businesses/search"
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {YELP_KEY}"
    }
    params_dict = {
        'latitude': latitude,
        'longitude': longitude,
        "radius": 1000,
        "limit": limit,
        "sort_by": "distance",
        "categories": "bars"
    }
    response = requests.get(url, headers=headers, params=params_dict).json()
    df = pd.json_normalize(response['businesses'])
    return df

In [ ]:
def nearby_venues_yelp(dataframe, limit):
    result_list = []

    for index, row in dataframe.iterrows():
        venues = get_venues_yelp(row['short_id'], row['latitude'], row['longitude'], limit)
        for _, venue in venues.iterrows():
            result_row = row.copy()
            for col in venues.columns:
                result_row[col] = venue[col]
            result_list.append(result_row)

    raw_result = pd.DataFrame(result_list)
    reduced_results_1 = raw_result[['short_id', 'name', 'id', 'review_count', 'rating', 'price', 'distance', 'location.address1', 'categories']]
    reduced_results_2 = reduced_results_1.copy()
    reduced_results_2['categories'] = reduced_results_1['categories'].apply(lambda x: x[0]['title'] if isinstance(x, list) and len(x) > 0 else None)
    reduced_results_2['price'] = reduced_results_1['price'].apply(lambda x: len(x) if pd.notnull(x) else x)
    reduced_results_2 = reduced_results_2.iloc[:, [0,2,1,8,7,6,5,4,3]]
    reduced_results_2.rename(columns={'short_id': 'bike_station_id',
                                    'id': 'yelp_id',
                                    'name': 'venue_name',
                                    'categories': 'category',
                                    'location.address1': 'address',
                                    'review_count': 'total_ratings'}, inplace=True)
    reduced_results_3 = reduced_results_2.reset_index(drop=True)
    return reduced_results_3

Put your parsed results into a DataFrame

In [9]:
yelp_dataframe = nearby_venues_yelp(bike_stations, 50)

In [10]:
yelp_dataframe

,bike_station_id,yelp_id,venue_name,category,address,distance,price,rating,total_ratings
0,4591,zLBmNC_uNqkeVZDvbGW7iw,Slackwater - Salt Lake City,Pizza,684 S 500 W,274.879042,2.0,4.6,562
1,4591,Cp83E-7HKrIGFfD5a4U3FQ,evo Hotel Crown Bar,Bars,660 S 400th W,287.568308,NaN,4.0,1
2,4591,78zR_lurEsQaE1fZRv7SOg,Melancholy,Wine Bars,556 S Gale St,330.742558,NaN,5.0,1
3,4591,XoN1n1_LgWzPmpJGqPCO-Q,Club X,Dance Clubs,445 S 400 W,386.442848,2.0,3.8,5
4,4591,oR5m2Yn5lRX22MLyJnMnUQ,Kiitos Brewing,Breweries,608 W 700th S,388.475851,2.0,4.6,141
...,...,...,...,...,...,...,...,...,...
2278,3423,kRRn8aWc_8FX-SoupV4HmQ,Sky SLC,Venues & Event Spaces,149 Pierpont Ave,581.468639,2.0,2.7,120
2279,3423,NxxMJl4nyelgy7w1ZK3juw,Trellis,Lounges,149 Pierpont Ave,581.468639,NaN,5.0,1
2280,3423,0YoRuIj79Tdqx-Nm8sCRAQ,Copper Common,Bars,111 E Broadway,586.271084,2.0,3.7,240
2281,3423,JfgI0_6DFtopbLxz1HZ5Ew,Bodega,New American,331 S Main St,593.054926,3.0,4.0,399


In [11]:
#yelp_dataframe.to_csv("../data/yelp_dataframe.csv", index=False)
#only uncomment the above line if you want to overwrite the csv file

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

As seen below, Yelp provides a more accurate representation on the reviews for all the different locations that it provides, since for the entire Yelp Dataframe, there is only 8.93% of the hits that don't provide any ratings at all (after converting ratings and total_ratings that have a 0 value to be considered null), compared to FourSquare, which lacks ratings on over 41.95% of the nearby hits provided for the bike stations.

In [51]:
yelp_dataframe_offline = pd.read_csv("../data/yelp_dataframe.csv")
fsq_dataframe_offline = pd.read_csv("../data/fsq_dataframe.csv")

In [ ]:
yelp_dataframe_offline_copy = yelp_dataframe_offline.copy()
yelp_dataframe_offline_copy['total_ratings'].replace(0, None, inplace=True)
yelp_dataframe_offline_copy['rating'].replace(0, None, inplace=True)

In [58]:
yelp_dataframe_offline_copy.info(), fsq_dataframe_offline.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2283 entries, 0 to 2282
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bike_station_id  2283 non-null   int64  
 1   yelp_id          2283 non-null   object 
 2   venue_name       2283 non-null   object 
 3   category         2283 non-null   object 
 4   address          2283 non-null   object 
 5   distance         2283 non-null   float64
 6   price            1267 non-null   float64
 7   rating           2079 non-null   object 
 8   total_ratings    2079 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 160.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1923 entries, 0 to 1922
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bike_station_id  1923 non-null   int64  
 1   fsq_id           1921 non-null   object 
 2   venue_name       1921 non-nul

(None, None)

Get the top 10 restaurants according to their rating

In [59]:
yelp_dataframe_offline_sorted = yelp_dataframe_offline.sort_values(by=['total_ratings', 'rating'], ascending=[False, False])
yelp_dataframe_offline_sorted.drop(columns=['bike_station_id', 'distance'], inplace=True)
yelp_dataframe_offline_sorted.drop_duplicates(inplace=True)
yelp_dataframe_offline_sorted.head(10)

,yelp_id,venue_name,category,address,price,rating,total_ratings
1675,f2TkM9bmBuSSo6N76E1Zog,Lucky 13,Burgers,135 W 1300th S,2.0,4.0,2171
91,pfyNG7R2Dcc0iSf5R1RaCg,Whiskey Street,New American,323 S Main St,2.0,4.1,1521
89,iC07-jSfyOWxKBqYH658kw,White Horse,Cocktail Bars,325 S Main St,2.0,4.5,1320
271,6kjoUUzT_l66Cgp9RcEJsQ,Slackwater Pub & Pizzeria,Pizza,209 24th Street,2.0,4.4,1172
26,98i7iJ00xIm0qDytcrQGMA,Zest Kitchen & Bar,Gluten-Free,275 S 200th W,2.0,4.3,1143
229,7GVAE88l_dPdB2FUsKbdbg,The Bayou,Cajun/Creole,645 S State St,2.0,4.0,1118
227,_WN2EffF-raS2qHSWXAp0A,Sapa Sushi Bar and Asian Grill,Sushi Bars,722 S State St,2.0,4.1,1032
19,i-5idxuSe775AJ1bLaWohQ,Caffé Molise,Italian,404 S West Temple,3.0,4.0,941
269,AlsbJp8mmOiGGp942VfH4Q,Sonora Grill,Mexican,2310 Kiesel Ave,2.0,3.9,797
34,x3bBfBbfsF_GGG1UZ5spGA,Gracie's,Bars,326 S West Temple St,2.0,3.2,727


In [34]:
fsq_dataframe_offline_sorted = fsq_dataframe_offline.sort_values(by=['total_ratings', 'rating'], ascending=[False, False])
fsq_dataframe_offline_sorted.drop(columns=['bike_station_id', 'distance'], inplace=True)
fsq_dataframe_offline_sorted.drop_duplicates(inplace=True)
fsq_dataframe_offline_sorted.head(10)

,fsq_id,venue_name,category,address,price,rating,total_ratings
68,4ae7198ef964a52067a821e3,Squatters Pub Brewery,Bar,147 W Broadway,2.0,8.7,757.0
1387,4b6e1b92f964a52003ab2ce3,Lucky 13,Bar,135 W 1300 S,2.0,8.8,386.0
568,4add36d0f964a520a46421e3,The Bayou,Pub,645 S State St,2.0,8.3,287.0
297,528954aa11d27852674d0669,Whiskey Street,Cocktail Bar,323 S Main St,2.0,8.6,270.0
62,4ae0f334f964a520f98321e3,Gracie's,Sports Bar,326 S West Temple,2.0,7.2,259.0
25,4aef5fccf964a520f1d721e3,BeerHive Pub & Grill,Beer Garden,128 S Main St,2.0,8.1,255.0
84,4aea79adf964a520b3bb21e3,The Green Pig Pub,Sports Bar,31 E 400 S,2.0,7.7,210.0
49,532b9ee611d29cb042b0af32,Beer Bar,Beer Bar,74 S Main St,2.0,8.8,166.0
14,4ad4f28ef964a5200afd20e3,Brewvies Cinema Pub,Indie Movie Theater,677 S 200 W,NaN,8.2,156.0
44,4bdbc82b63c5c9b6f4ab2968,Bar X,Night Club,155 E 200 S,2.0,7.6,154.0
